In [1]:
import sys
import os 
os.chdir('..')
sys.path.append("./scripts")
import numpy as np
import config
import torch
import data_collection.BLong_preamble_detection as pd
import utils
import matplotlib.pyplot as plt
from scramble_table import *
import DANF.blong_dnn as dnn
import raw_data_preprocessing as rdp
import opt.goodput_opt as go

f = None

cur_data = None
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
power_of_2 = np.array([1, 2, 4, 8, 16, 32, 64, 128], dtype=np.uint8)
target_aa_bit = np.array([0,0,0,1,1,0,1,1,0,1,1,1,1,1,0,1,1,0,0,1,0,0,0,1,0,1,1,1,0,0,0,1], dtype=np.uint8)
preamble_bits = np.array([0, 1, 0, 1, 0, 1, 0, 1], dtype=np.uint8)
header_bits = np.array([0,0,0,0,1,1,1,0], dtype=np.uint8)
len_bits = np.array([1,1,1,1,1,1,1,1], dtype=np.uint8)

def deal_outliers(pbcrs, bit_num):
    mean_bcrs = []
    for i in range(pbcrs.shape[0]):
        bcrs = pbcrs[i, :]
        q1, q3 = np.percentile(bcrs, [25, 75])
        iqr = 1.5 * (q3 - q1)
        upper = q3 + iqr
        lower = q1 - iqr
        valid_idx = np.where((bcrs<=upper)&(bcrs>=lower))[0]
        mean_bcr = np.mean(bcrs[valid_idx]) / bit_num
        mean_bcrs.append(mean_bcr)
    return mean_bcrs

def case_study_combine(raw_file, extf, file_nums):
    best_steps = {1:5000, 2:5600, 4:4900, 8:6200, 16:5000, 32:5200, 64:3600}
    if extf <= 16:
        num = 265
    else:
        num = 100
    if extf > 1:
        data_bits = np.concatenate([target_aa_bit, header_bits, rand_bits])
        data_bits = data_bits[:(num-1)*8]
    else:
        data_bytes = np.concatenate([np.array([0x70, 0xff], dtype=np.uint8), scramble_table[0, :255]])
        data_bytes = data_bytes ^ scramble_table[8, :257]
        data_bytes = np.concatenate([data_bytes, np.array([224, 247, 242], dtype=np.uint8)])
        data_bits = utils.bytes_to_bits(data_bytes)
        data_bits = np.concatenate([target_aa_bit, data_bits])
        # print(data_bytes)
    packets = []
    for n in file_nums:
        raw_data = np.load("./raw_data/" + raw_file.format(extf, n) + ".npz")
        raw_signal_ori = raw_data["arr_0"].flatten()[np.newaxis, :]
        raw_signal = utils.filter(raw_signal_ori)
        thres = 90
        p_idx = pd.preamble_detection_aa_check(raw_signal_ori, raw_signal, extf, thres_percent=thres, full=True)
        p_idx = p_idx[0]
        # Then we start to slice the data bytes
        if extf > 1:
            bits_pre_pdu = int(2040 // extf)
        else:
            bits_pre_pdu = 265 * 8
        res_len = 2040 - bits_pre_pdu * extf
        skip_len = (80 + 150 + res_len) * config.sample_pre_symbol

        done_flag = False
        print("{} has {} packets".format(n, len(p_idx)))
        for i in range(len(p_idx)):
            pkt = []
            ptr = 8 * extf * config.sample_pre_symbol + p_idx[i]
            for j in range(8, num*8):
                if j % bits_pre_pdu == 0:
                    ptr += skip_len
                n_ptr = ptr + extf * config.sample_pre_symbol
                sp = ptr - config.extra
                ep = n_ptr + config.extra
                if ep >= raw_signal_ori.shape[1]:
                    done_flag = True
                    break
                pkt.append(raw_signal_ori[0, sp:ep].copy())
                ptr = n_ptr
            if done_flag:
                break
            packets.append(pkt)
        del raw_data, raw_signal
    packets = np.array(packets)
    labels = np.concatenate([data_bits[:(num-1)*8] for _ in range(packets.shape[0])])
    packets = packets.reshape([-1, packets.shape[2]])
    labels = np.concatenate([data_bits[:(num-1)*8] for _ in range(packets.shape[0])])
    bcrs = []

    for dextf in [1, 2, 4, 8, 16, 32, 64]:
        dext_syms, _, _, dext_labels = rdp.save_dext_slices(packets, None, None, labels, extf, dextf, raw_file.format(dextf, 0), 0.2)
        model_name = "dnn_wr_upd_{}e_{}s.pt".format(dextf, best_steps[dextf])
        bits, correct_num = dnn.decode_ber(dext_syms, dext_labels, dextf, model_name)
        bcr = correct_num / dext_syms.shape[0]
        bcrs.append(bcr)
        print(raw_file, dextf, bcrs[-1])
    return np.array(bcrs)

def case_study_process(raw_file, extf, load=True):
    best_steps = {1:5000, 2:5600, 4:4900, 8:6200, 16:5000, 32:5200, 64:3600}
    if extf <= 16:
        num = 265
    else:
        num = 100
    if extf > 1:
        data_bits = np.concatenate([target_aa_bit, header_bits, rand_bits])
        data_bits = data_bits[:(num-1)*8]
    else:
        data_bytes = np.concatenate([np.array([0x70, 0xff], dtype=np.uint8), scramble_table[0, :255]])
        data_bytes = data_bytes ^ scramble_table[8, :257]
        data_bytes = np.concatenate([data_bytes, np.array([224, 247, 242], dtype=np.uint8)])
        data_bits = utils.bytes_to_bits(data_bytes)
        data_bits = np.concatenate([target_aa_bit, data_bits])
        # print(data_bytes)
    if not load:
        # print(gt_p_time[1:] - gt_p_time[0:-1])
        raw_data = np.load("./raw_data/" + raw_file.format(extf) + ".npz")
        raw_signal_ori = raw_data["arr_0"].flatten()[np.newaxis, :]
        raw_signal = utils.filter(raw_signal_ori)
        thres = 9
        # p = utils.get_phase_diff(raw_signal)
        # pa = pd.get_preamble_aa_mask(extf)
        # pa_corr = pd.cal_corrlation(p, pa)[0]
        # p_diff = utils.get_phase_cum(raw_signal)
        # print(np.max(pa_corr))
        # plt.figure(figsize=(12, 18))
        # plt.subplot(311)
        # plt.plot(pa_corr)
        # plt.subplot(312)
        # plt.plot(raw_signal[0].real)
        # plt.plot(raw_signal[0].imag)
        # plt.subplot(313)
        # plt.plot(p_diff[0][:])
        # plt.savefig("./figs/pa_corr2.pdf")
        # plt.close()
        p_idx = pd.preamble_detection_aa_check(raw_signal_ori, raw_signal, extf, thres_percent=thres, full=True)
        p_idx = p_idx[0]
        # Then we start to slice the data bytes
        if extf > 1:
            bits_pre_pdu = int(2040 // extf)
        else:
            bits_pre_pdu = 265 * 8
        res_len = 2040 - bits_pre_pdu * extf
        skip_len = (80 + 150 + res_len) * config.sample_pre_symbol
        packets = []
        f_packets = []
        done_flag = False
        print("has {} packets".format(len(p_idx)))
        for i in range(len(p_idx)):
            pkt = []
            f_pkt = []
            ptr = 8 * extf * config.sample_pre_symbol + p_idx[i]
            for j in range(8, num*8):
                if j % bits_pre_pdu == 0:
                    ptr += skip_len
                n_ptr = ptr + extf * config.sample_pre_symbol
                sp = ptr - config.extra
                ep = n_ptr + config.extra
                if ep >= raw_signal_ori.shape[1]:
                    done_flag = True
                    break
                pkt.append(raw_signal_ori[0, sp:ep])
                f_pkt.append(raw_signal[0, sp:ep])
                ptr = n_ptr
            if done_flag:
                break
            packets.append(pkt)
            f_packets.append(f_pkt)
        packets = np.array(packets)
        # f_packets = np.array(f_packets) 
        np.savez("./processed_data/" + raw_file + ".npz", f_packets=f_packets, packets=packets)
    else:
        data = np.load("./processed_data/" + raw_file + ".npz")
        # f_packets = data["f_packets"]
        packets = data["packets"]
    if packets.shape[0] == 0:
        return np.zeros(3), np.zeros(3), 1
    packets = packets.reshape([-1, packets.shape[2]])
    # packets = utils.signal_normalize(packets, 1)
    # f_packets = utils.filter(packets).astype(np.complex64)
    labels = np.concatenate([data_bits[:(num-1)*8] for _ in range(packets.shape[0])])
    bcrs = []
# 2, 4, 8, 16, 32, 64
    for dextf in [64]:
        dext_syms, _, _, dext_labels = rdp.save_dext_slices(packets, None, None, labels, extf, dextf, raw_file.format(dextf), 1)
        model_name = "dnn_wr_upd_{}e_{}s.pt".format(dextf, best_steps[dextf])
        bits, correct_num = dnn.decode_ber(dext_syms, dext_labels, dextf, model_name)
        bcr = correct_num / dext_syms.shape[0]
        bcrs.append(bcr)
        print(raw_file, dextf, bcrs[-1])
    return np.array(bcrs)



def get_dis_ber(extf=8):
    bcrs = []
    bcrs_woupd = []
    dis = np.array([10, 30, 50, 70, 90, 110])
    for d in dis:
        data = np.load("./processed_data/case_study/outdoor/{}m_18c_{}e_100n_0.npz".format(d, extf))
        sym = data["sym"]
        label = data["label"]
        num = int(0.05 * sym.shape[0])
        syms = sym[num:, :].copy()
        labels = label[num:].copy()
        model_name = "dnn_uni_cpy_wr_dis_{}e_{}s.pt".format(extf, 4900)
        bits, correct_num = dnn.decode_ber(syms, labels, extf, model_name)
        bcr = correct_num / syms.shape[0]
        bcrs.append(bcr)
        print(d, bcr)
        model_name = "dnn_uni_cpy_wr_{}e_{}s.pt".format(extf, 6600)
        bits, correct_num = dnn.decode_ber(syms, labels, extf, model_name)
        bcr = correct_num / syms.shape[0]
        bcrs_woupd.append(bcr)
        print(d, bcr)
    bcrs = np.array(bcrs)
    bcrs_woupd = np.array(bcrs_woupd)
    print(bcrs)
    bcrs = np.hstack([dis.reshape([-1, 1]), bcrs])
    bcrs_woupd = np.hstack([dis.reshape([-1, 1]), bcrs_woupd])
    # np.savetxt("./output/distance_ber_extf={}.csv".format(extf), bcrs, delimiter=",")
    # np.savetxt("./output/distance_ber_woupd_extf={}.csv".format(extf), bcrs_woupd, delimiter=",")

In [2]:
print(os.getcwd())
get_dis_ber()

/liymdata/liym/BLong_sensys


10 [1. 1. 1.]
10 [1. 1. 1.]
30 [0.99947895 0.99946316 0.99942632]
30 [0.99958421 0.99946316 0.99942632]
50 [0.99848947 0.99632632 0.99538947]
50 [0.9987     0.99632632 0.99538947]
70 [0.99372105 0.9648     0.95632105]
70 [0.99070526 0.9648     0.95632105]
90 [0.98681053 0.94844211 0.93965789]
90 [0.98221053 0.94844211 0.93965789]
110 [0.87741053 0.81082632 0.79525789]
110 [0.87558947 0.81082632 0.79525789]
[[1.         1.         1.        ]
 [0.99947895 0.99946316 0.99942632]
 [0.99848947 0.99632632 0.99538947]
 [0.99372105 0.9648     0.95632105]
 [0.98681053 0.94844211 0.93965789]
 [0.87741053 0.81082632 0.79525789]]
